**Задание Light**
1. Выберите произвольный документ для дообучения chatGPT (инструкция, регламент, правила и т.п.)
2. Обучите chatGPT на своей базе
3. Отправьте вопросы для тестирования созданной схемы

In [27]:
%%capture
%pip install -q faiss-cpu==1.7.4 openai==1.6.1 tiktoken==0.5.2 langchain==0.2.1
%pip install -qU langchain-openai==0.0.4
%pip install -U langchain-community==0.2.1
!rm -r sample_data

In [28]:
%pip install langchain-core==0.1.35
%pip install -U langchain langchain-openai openai

  Using cached langchain_core-0.1.35-py3-none-any.whl.metadata (6.0 kB)
Using cached langchain_core-0.1.35-py3-none-any.whl (273 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.2.1 requires langchain-core<0.3.0,>=0.2.0, but you have langchain-core 0.1.35 which is incompatible.
langchain 0.2.1 requires langchain-core<0.3.0,>=0.2.0, but you have langchain-core 0.1.35 which is incompatible.
langchain-text-splitters 0.2.4 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.1.35 which is incompatible.
  Using cached langchain-0.3.24-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_openai-0.3.14-py3-none-any.whl.metadata (2.3 kB)

In [1]:
# Импорт функции getpass для безопасного ввода пароля
import getpass

# Импорт библиотеки openai для взаимодействия с OpenAI API
import openai

# Импорт модуля os для работы с операционной системой
import os

# Запрос на ввод API ключа для OpenAI с помощью getpass, чтобы скрыть вводимые символы
openai_key = getpass.getpass("OpenAI API Key:")

# Установка переменной среды OPENAI_API_KEY для передачи ключа в переменную среды
os.environ["OPENAI_API_KEY"] = openai_key

# Установка API ключа для библиотеки openai
openai.api_key = openai_key

# Выбор модели
model_LLM = "gpt-3.5-turbo"

OpenAI API Key:··········


In [2]:
# Импорт класса OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

# Импорт класса CharacterTextSplitter из модуля text_splitter в библиотеке langchain
from langchain.text_splitter import CharacterTextSplitter

# Импорт модуля FAISS из vectorstores в библиотеке langchain
from langchain.vectorstores import FAISS

# Импорт класса TextLoader из модуля document_loaders в библиотеке langchain
from langchain.document_loaders import TextLoader

# Импорт модуля os для работы с операционной системой
import os

# Импорт модуля gdown для загрузки файлов из интернета через Google Drive API
import gdown

import requests

In [3]:
# Загружаем базу

url = 'https://docs.google.com/document/d/14MyGYfrRRKC3mB92Yukqy5whXZ6LYkWi/edit?usp=sharing&ouid=101912731247060282743&rtpof=true&sd=true'
doc_match = url.split('/')[-2]
print(doc_match)

14MyGYfrRRKC3mB92Yukqy5whXZ6LYkWi


In [4]:
# Читаем текст и сохраняем его в TXT файл

response = requests.get(f'https://docs.google.com/document/d/{doc_match}/export?format=txt')
response.raise_for_status()
text = response.text

with open("/content/Инструкция_первая_помощь.txt", 'w', encoding= 'utf-8') as f:
    f.write(text)

In [5]:
# Создание экземпляра класса TextLoader с указанием пути к файлу с текстом для загрузки
loader = TextLoader("/content/Инструкция_первая_помощь.txt")

# Загрузка текстовых документов из указанного файла с помощью метода load() объекта loader
documents = loader.load()

In [6]:
type(documents[0])

langchain_core.documents.base.Document

In [7]:
documents[0].page_content

'\ufeffИНСТРУКЦИЯ ОТ - 14\nпо оказанию первой помощи при несчастных случаях на производстве\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n2022\n1 Общая часть\nВремя от момента травмы (отравления), до получения помощи должно быть предельно сокращено. Человек, оказывающий помощь обязан действовать решительно, целесообразно. Необходимо принять меры  к прекращению воздействия повреждающих факторов и правильно оценить состояние пострадавшего. При осмотре пострадавшего сначала определяют тяжесть поражения. Нужно выявить, устранить поражения (кровотечение, остановку дыхания и сердечной деятельности, нарушение проходимости дыхательных путей, сильную головную боль).\n        Во всех случаях при оказании первой медицинской помощи необходимо доставить пострадавшего в Лечебное Профилактическое Учреждение (ЛПУ) и вызов не должен приостанавливать оказание первой медицинской помощи.\n\n\nПервая помощь – это совокупность простых, ц

In [8]:
# Создание экземпляра класса CharacterTextSplitter с заданными параметрами:
# - separator: строка, используемая для разделения текста на части
# - chunk_size: размер части текста, который будет обрабатываться за один раз
# - chunk_overlap: количество символов, на которое перекрываются соседние части текста
character = CharacterTextSplitter(separator=" ", chunk_size=1_000, chunk_overlap=0)

# Запускаем разделение по чанкам
docs = character.split_documents(documents)

In [9]:
print(f'Всего фрагментов: {len(docs)}')

Всего фрагментов: 28


In [10]:
docs[0]

Document(metadata={'source': '/content/Инструкция_первая_помощь.txt'}, page_content='\ufeffИНСТРУКЦИЯ ОТ - 14\nпо оказанию первой помощи при несчастных случаях на производстве\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n2022\n1 Общая часть\nВремя от момента травмы (отравления), до получения помощи должно быть предельно сокращено. Человек, оказывающий помощь обязан действовать решительно, целесообразно. Необходимо принять меры к прекращению воздействия повреждающих факторов и правильно оценить состояние пострадавшего. При осмотре пострадавшего сначала определяют тяжесть поражения. Нужно выявить, устранить поражения (кровотечение, остановку дыхания и сердечной деятельности, нарушение проходимости дыхательных путей, сильную головную боль).\n Во всех случаях при оказании первой медицинской помощи необходимо доставить пострадавшего в Лечебное Профилактическое Учреждение (ЛПУ) и вызов не должен приостанавливать оказание

In [11]:
n = 0

print(f'\033[91mДлина: {len(docs[n].page_content)}\033[0m')
print('\033[92m=\033[0m'*20)

print(docs[n].page_content)

Длина: 992
﻿ИНСТРУКЦИЯ ОТ - 14
по оказанию первой помощи при несчастных случаях на производстве


























































2022
1 Общая часть
Время от момента травмы (отравления), до получения помощи должно быть предельно сокращено. Человек, оказывающий помощь обязан действовать решительно, целесообразно. Необходимо принять меры к прекращению воздействия повреждающих факторов и правильно оценить состояние пострадавшего. При осмотре пострадавшего сначала определяют тяжесть поражения. Нужно выявить, устранить поражения (кровотечение, остановку дыхания и сердечной деятельности, нарушение проходимости дыхательных путей, сильную головную боль).
 Во всех случаях при оказании первой медицинской помощи необходимо доставить пострадавшего в Лечебное Профилактическое Учреждение (ЛПУ) и вызов не должен приостанавливать оказание первой медицинской помощи.


Первая помощь – это совокупность простых, целесообразных мер по охране здоровья и жизни пострадавшего от травмы 

In [12]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(docs, embeddings)

In [13]:
from langchain_openai import OpenAIEmbeddings


In [14]:
query = "Какие виды первой помощи оказываются?"
results = db.similarity_search(query, k=4)

for i in range(4):
  print(f"\033[91m**отрезок №{i+1}**\033[0m\n",results[i].page_content)


**отрезок №1**
 внезапно заболевшего человека. 
Первую помощь оказывают на месте происшествия, еще до прихода врача или до транспортировки пострадавшего в больницу. Правильно оказанная первая помощь сокращает время лечения, способствует быстрейшему заживлению ран и часто является решающим фактором при спасении жизни.
Первая помощь включает в себя:
1. Эвакуацию пострадавшего с места происшествия или немедленное освобождение от воздействия опасного фактора;
 Обеспечения вызова спасателей или бригады скорой медицинской помощи;
2. Разрешения на оказание первой помощи (в случае если пострадавший находится в сознании);
3. Оказание непосредственно первой помощи пострадавшему, которая включает в себя оценку состояния пострадавшего, осмотр и оказание помощи в зависимости от состояния и характера повреждений;
4. Обеспечение транспортировки в лечебное учреждение (в случаях, когда нет возможности вызвать бригаду спасателей или скорой помощи, или их подъезд к месту происшествия невозможен).
Основны

In [15]:
import re
import requests
import openai
from langchain.docstore.document import Document

In [16]:
# Загрузка базы
url = 'https://docs.google.com/document/d/14MyGYfrRRKC3mB92Yukqy5whXZ6LYkWi/edit?usp=sharing&ouid=101912731247060282743&rtpof=true&sd=true'
match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
print(match_)

<re.Match object; span=(23, 68), match='/document/d/14MyGYfrRRKC3mB92Yukqy5whXZ6LYkWi'>


In [17]:
doc_id = match_.group(1)
print(doc_id)

14MyGYfrRRKC3mB92Yukqy5whXZ6LYkWi


In [18]:
response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
response.raise_for_status()
text = response.text

In [19]:
print(len(text.split()))

3502


In [20]:
system = """Ты специалист по оказанию первой помощи. У тебя есть большой документ с инструкцией по оказанию первой помощи.
Тебе задает вопрос сотрудник, дай ему ответ, опираясь на документ, постарайся ответить так, чтобы человек сумел оказать первую помощь. и отвечай максимально точно по документу,
не придумывай ничего от себя. Не упоминай документ при ответе, клиент ничего не должен знать о документе, по которому ты отвечаешь."""

In [21]:
question = "как оказывать помощь человеку попавшему под действие электрического тока?"

In [22]:
docs_score = db.similarity_search_with_score(question, k=4)
docs_score

[(Document(metadata={'source': '/content/Инструкция_первая_помощь.txt'}, page_content='патологическое состояние, обусловленное воздействием на пострадавшего электрического тока.\n\n\n При термических ожогах необходимо:\n1. Вывести (вынести) пострадавшего из зоны огня.\n2. Снять горящую одежду или залить водой.\n3. Уложить пострадавшего, успокоить.\n4. Вызвать скорую помощь.\n5. Дать обильное питье 1-1,5 литра, при сохранении сознании.\n6. Наложить стерильную повязку, пузыри не вскрывать.\n\n\nПри химических ожогах необходимо:\n1. Ожоговую поверхность промыть проточной водой в течение 15-20 минут.\n2. На ожоговую поверхность наложить сухую стерильную повязку.\n3. Вызвать скорую помощь.\n\n\n При электрическом ожоге необходимо:\n1. Соблюдать правила личной безопасности (не трогать источник голыми руками, не подходить к пострадавшему, если он лежит на влажной почве).\n2. Прекратить контакт с токонесущим предметом (обесточить пострадавшего).\n3. Вызвать скорую помощь.\n4. Пострадавшего уло

In [23]:
for score in docs_score:
  print(score[1])

0.2996763
0.3114975
0.32095915
0.32699165


In [24]:
# Поиск 4 отрывков в базе данных, содержание которых подходит для ответа на вопрос
docs = db.similarity_search(question, k=4)

message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\033[91m\nОтрывок документа №{i+1}\n=====================\n\033[0m' + doc.page_content + '\n\n' for i, doc in enumerate(docs)]))
print(message_content)


Отрывок документа №1
патологическое состояние, обусловленное воздействием на пострадавшего электрического тока. 
 При термических ожогах необходимо:
1. Вывести (вынести) пострадавшего из зоны огня.
2. Снять горящую одежду или залить водой.
3. Уложить пострадавшего, успокоить.
4. Вызвать скорую помощь.
5. Дать обильное питье 1-1,5 литра, при сохранении сознании.
6. Наложить стерильную повязку, пузыри не вскрывать. 
При химических ожогах необходимо:
1. Ожоговую поверхность промыть проточной водой в течение 15-20 минут.
2. На ожоговую поверхность наложить сухую стерильную повязку.
3. Вызвать скорую помощь. 
 При электрическом ожоге необходимо:
1. Соблюдать правила личной безопасности (не трогать источник голыми руками, не подходить к пострадавшему, если он лежит на влажной почве).
2. Прекратить контакт с токонесущим предметом (обесточить пострадавшего).
3. Вызвать скорую помощь.
4. Пострадавшего уложить, успокоить.
5. Определить наличие сознания.
6. Измерить пульс, проверить дыхание, сер

In [25]:
# Делаем запрос с подачей найденных в базе отрывков
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": f"Документ с информацией для ответа сотруднику: {message_content}\n\nВопрос сотрудника: \n{question}"}
]

completion = openai.chat.completions.create(
    model= model_LLM,
    messages=messages,
    temperature=0
)

In [26]:
answer = completion.choices[0].message.content

temp = answer.split('.')
temp = map(lambda x: x.strip(), temp)
ans = '.\n'.join(temp)
print(ans)

При оказании помощи человеку, попавшему под действие электрического тока, необходимо соблюдать следующие шаги:
1.
Соблюдать правила личной безопасности, не трогать источник тока голыми руками и не подходить к пострадавшему, если он лежит на влажной почве.
2.
Прекратить контакт с токонесущим предметом, обесточить пострадавшего.
3.
Вызвать скорую помощь.
4.
Уложить пострадавшего, успокоить.
5.
Определить наличие сознания.
6.
Измерить пульс, проверить дыхание, сердцебиение.
7.
На местах входа и выхода тока (прострелах) наложить сухую повязку.
8.
При незначительном ударе грозы или воздействии электротока, госпитализировать пострадавшего, так как возможна остановка сердца.

